In [1]:
import sys
sys.path.append('../')

from rocket_rag.utils import *
from rocket_rag.agent import RagAgent

In [2]:
load = '40kg'

In [3]:
if_files_dict = parse_files(main_directory=INFERENCE_DIR)
if_ts_files = if_files_dict[load]

In [4]:
# np.random.seed(37)
rand_idx = np.random.randint(0, len(if_ts_files))
if_ts_filename = if_ts_files[rand_idx]
print(f'Randomly selected file: {if_ts_filename}')

if_rocket_feature = fit_transform(ts_filename=[if_ts_filename],
                                  field='current',
                                  smooth=True,
                                  smooth_ws=15,
                                  tolist=False,
                                  verbo=True)

2024-05-29 22:45:46.816 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points
2024-05-29 22:45:46.822 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-29 22:45:46.823 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Randomly selected file: ../data/inference/40kg\lackLubrication1\lackLubrication1_40_9_2.csv


In [5]:
agent = RagAgent(vs_dir=f'../store/nodes_{load}.pkl')

2024-05-29 22:45:51.298 | DEBUG    | rocket_rag.node_indexing:load_node_indexing:98 - Loading all nodes...
2024-05-29 22:45:51.863 | INFO     | rocket_rag.node_indexing:load_node_indexing:102 - All nodes are loaded.
2024-05-29 22:45:52.205 | INFO     | rocket_rag.agent:_init_openai_client:67 - OpenAI API key found. OpenAI client Initialized.
2024-05-29 22:45:52.250 | INFO     | rocket_rag.agent:__init__:59 - RAG agent initialized.


In [6]:
rocket_feature = agent.get_rocket_feature(if_ts_filename)
print(f'Rocket feature shape: {rocket_feature.shape}')
fault_prediction, score = agent.get_fault_prediction()
print(f'Fault prediction: {fault_prediction}, Score: {score}')

2024-05-29 22:45:52.259 | DEBUG    | rocket_rag.utils:fit_transform:149 - Extract the time series data points
2024-05-29 22:45:52.271 | INFO     | rocket_rag.utils:fit:125 - time series extracted SUCCESSFULLY.
2024-05-29 22:45:52.272 | DEBUG    | rocket_rag.utils:fit_transform:161 - Rocket transforming...


Rocket feature shape: (1, 20000)
Fault prediction: ['lackLubrication1_40_5_4'], Score: 1.0


In [7]:
agent.generate_fault_diagnosis_statement()
print(agent.fault_diagnosis_json)

{'fault_type': 'lackLubrication', 'degradation_level': 1, 'retrieval_result(s)': 'lackLubrication1_40_5_4', 'score': 1.0, 'distances': [], 'description': 'The actuator is operating with insufficient lubrication, leading to increased friction and potential for wear and tear, impacting performance and longevity.'}


In [8]:
agent.refine_fault_diagnosis_statement()
print(agent.refined_diagnosis_json)

{'fault_type': 'lackLubrication', 'degradation_level': 1, 'refinement': 'Fault in lacklubrication'}


In [9]:
agent.generate_multi_queries()
print(agent.generated_queries)

['lack of lubrication in linear actuators repair', 'how to fix lack of lubrication in linear actuators', 'linear actuator maintenance for lack of lubrication issue', 'recovery methods for lack of lubrication in linear actuators', 'troubleshooting lack of lubrication in linear actuators']


In [10]:
print(agent.tools.get_available_tools())

{'call_google': <function Tools.call_google at 0x000001F1F0AF77E0>, 'save_to_file': <function Tools.save_to_file at 0x000001F1F0AF7880>, 'get_available_tools': <function Tools.get_available_tools at 0x000001F1F0AF7920>}


In [11]:
agent.call_google_search()
# print(agent.query_answers)

0it [00:00, ?it/s]

2024-05-29 22:45:58.370 | INFO     | rocket_rag.agent:call_google_search:283 - Calling Google search API for query 1/5
2024-05-29 22:46:02.828 | INFO     | rocket_rag.agent:call_google_search:283 - Calling Google search API for query 2/5
2024-05-29 22:46:06.384 | INFO     | rocket_rag.agent:call_google_search:283 - Calling Google search API for query 3/5
2024-05-29 22:46:10.615 | INFO     | rocket_rag.agent:call_google_search:283 - Calling Google search API for query 4/5
2024-05-29 22:46:16.270 | INFO     | rocket_rag.agent:call_google_search:283 - Calling Google search API for query 5/5


["I found some relevant information regarding lack of lubrication in linear actuators repair. Here are a few snippets from the search results:\n\n1. Ensure the right type of lubricant is used depending on the actuator's construction.\n2. Detect and classify states of a linear actuator: normal, backlash, lack of lubrication, and spalling.\n3. Proper lubrication is crucial as lack of lubrication can cause strange noises.\n4. Lack of appropriate lubrication is the number one failure mode in roller screws and roller screw actuators.\n\nIf you need more detailed information or specific instructions, please let me know!",
 'I found some information related to fixing the lack of lubrication in linear actuators. Here are some snippets from the search results:\n\n1. Make sure the actuator is properly lubricated as lack of lubrication can cause strange noises.\n2. Detect and classify states of a linear actuator: normal, backlash, lack of lubrication, and spalling.\n3. It may not be possible to s

In [12]:
await agent.gather_query_answers()

2024-05-29 22:46:20.262 | INFO     | rocket_rag.agent:gather_query_answers:354 - Start to generate the summerization for 2 nodes...
2024-05-29 22:46:22.854 | INFO     | rocket_rag.agent:gather_query_answers:363 - Combined into 2 responses, keep combining
2024-05-29 22:46:22.858 | INFO     | rocket_rag.agent:gather_query_answers:354 - Start to generate the summerization for 1 nodes...
2024-05-29 22:46:27.116 | INFO     | rocket_rag.agent:gather_query_answers:360 - Combined all responses to one. Done


"The lack of lubrication in linear actuators is a critical issue that can lead to various problems such as strange noises and equipment failure. Proper lubrication is essential, and using the right type of lubricant based on the actuator's construction is crucial. Detecting and classifying the actuator's state, including lack of lubrication, is vital for maintenance. Lack of appropriate lubrication is the primary failure mode in roller screws and roller screw actuators. Establishing a maintenance schedule, avoiding overgreasing, and understanding the importance of lubrication in separating machine components are key steps to prevent lubrication-related issues.\n\nRecovery methods for lack of lubrication in linear actuators include troubleshooting steps. Topics covered include compact and light-weight spring cylinder linear actuators, flushing saltwater to prevent harm to motors, valve and actuator types, lubricant depletion analysis, temperature issues due to lack of lubrication, dry f

In [13]:
agent.parse_output_contents()

2024-05-29 22:46:27.140 | DEBUG    | rocket_rag.agent:parse_output_contents:402 - Parsing the output file with provided format...


'# FAULT DIAGNOSIS REPORT\n\n## SYSTEM INFORMATION\n- Fault Type: Fault Type Diagnosis: lackLubrication\n- Fault Level: 1\n- Fault Refinement: Fault in lacklubrication\n- Fault Description: The actuator is operating with insufficient lubrication, leading to increased friction and potential for wear and tear, impacting performance and longevity.\n\n## SEARCH QUERY\n1. I found some relevant information regarding lack of lubrication in linear actuators repair. Here are a few snippets from the search results:\n   - Ensure the right type of lubricant is used depending on the actuator\'s construction.\n   - Detect and classify states of a linear actuator: normal, backlash, lack of lubrication, and spalling.\n   - Proper lubrication is crucial as lack of lubrication can cause strange noises.\n   - Lack of appropriate lubrication is the number one failure mode in roller screws and roller screw actuators.\n\n2. I found some information related to fixing the lack of lubrication in linear actuato

In [14]:
agent.save_output_contents(file_name='fault-diagnosis')

2024-05-29 22:46:45.297 | INFO     | rocket_rag.agent:save_output_contents:425 - Parse the file to the ../logs/fault-diagnosis-2024-05-29_22-46-45.md
2024-05-29 22:46:45.298 | INFO     | rocket_rag.agent:save_output_contents:431 - File fault-diagnosis is parsed and stored successfully.
